In [1]:
"""
This examples trains a CrossEncoder for the NLI task. A CrossEncoder takes a sentence pair
as input and outputs a label. Here, it learns to predict the labels: "contradiction": 0, "entailment": 1, "neutral": 2.

It does NOT produce a sentence embedding and does NOT work for individual sentences.

Usage:
python training_nli.py
"""
from torch.utils.data import DataLoader
import math
from sentence_transformers import LoggingHandler, util
from sentence_transformers.cross_encoder import CrossEncoder
from sentence_transformers.cross_encoder.evaluation import CEBinaryAccuracyEvaluator, CEBinaryClassificationEvaluator
from sentence_transformers.readers import InputExample
import logging
from datetime import datetime
import os
import gzip
import csv

#### Just some code to print debug information to stdout
logging.basicConfig(format='%(asctime)s - %(message)s',
                    datefmt='%Y-%m-%d %H:%M:%S',
                    level=logging.INFO,
                    handlers=[LoggingHandler()])
logger = logging.getLogger(__name__)
#### /print debug information to stdout

data_path = './dureader/train/cross.train.tsv'
model_name = 'output/cross-encoder_hlf-rbtl3-2022-06-11'

train_batch_size = 64
num_epochs = 6
model_save_path = 'output/cross-encoder_hlf-rbtl3'+'-'+datetime.now().strftime("%Y-%m-%d")

In [2]:
model = CrossEncoder(model_name)

2022-06-11 02:56:31 - Use pytorch device: cuda


In [3]:
train_samples = []
with open(data_path, 'r', encoding='utf-8') as f:
    for line in f.readlines():
        data = line.rstrip().split('\t')
        inp_example = InputExample(texts=[data[0][:256], data[2][:256]], label=float(data[3]))
        train_samples.append(inp_example)

In [4]:
from sklearn.model_selection import train_test_split

train_samples, test_samples = train_test_split(train_samples, test_size=0.01, random_state=13)
train_samples, dev_samples = train_test_split(train_samples, test_size=0.01, random_state=13)

print("Train sentences:", len(train_samples))
print("Dev sentences:", len(dev_samples))
print("Test sentences:", len(test_samples))

Train sentences: 1089846
Dev sentences: 11009
Test sentences: 11120


In [5]:
#We wrap train_samples, which is a list ot InputExample, in a pytorch DataLoader
train_dataloader = DataLoader(train_samples, shuffle=True, batch_size=train_batch_size)

#During training, we use CESoftmaxAccuracyEvaluator to measure the accuracy on the dev set.
evaluator = CEBinaryAccuracyEvaluator.from_input_examples(dev_samples, name='dev')

warmup_steps = math.ceil(len(train_dataloader) * num_epochs * 0.05) #10% of train data for warm-up
logger.info("Warmup-steps: {}".format(warmup_steps))

2022-06-11 02:56:40 - Warmup-steps: 5109


In [ ]:
# Train the model
model.fit(train_dataloader=train_dataloader,
          optimizer_params={'lr': 3e-05}, 
          evaluator=evaluator,
          epochs=num_epochs,
          evaluation_steps=10000,
          warmup_steps=warmup_steps,
          output_path=model_save_path, 
          use_amp=True)

model.save(model_save_path)

/root/miniconda3/lib/python3.8/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch:   0%|          | 0/6 [00:00<?, ?it/s]

Iteration:   0%|          | 0/17029 [00:00<?, ?it/s]

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
/root/miniconda3/lib/python3.8/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:198: FutureWarning: Non-finite norm encountered in torch.nn.utils.clip_grad_norm_; continuing anyway. Note that the default behavior will change in a future release to error out if a non-finite total norm is encountered. At that point, setting error_if_nonfinite=false will be required to retain the old behavior.
  torch.nn.utils.clip_grad_norm_(self.model.parameters(), max_grad_norm)


2022-06-11 03:22:47 - CEBinaryAccuracyEvaluator: Evaluating the model on dev dataset in epoch 0 after 10000 steps:
2022-06-11 03:22:59 - Accuracy: 83.75
2022-06-11 03:22:59 - Save model to output/cross-encoder_hlf-rbtl3-2022-06-11
2022-06-11 03:41:23 - CEBinaryAccuracyEvaluator: Evaluating the model on dev dataset after epoch 0:
2022-06-11 03:41:35 - Accuracy: 83.60


...

2022-06-11 05:38:15 - CEBinaryAccuracyEvaluator: Evaluating the model on dev dataset in epoch 3 after 10000 steps:
2022-06-11 05:38:27 - Accuracy: 83.36
2022-06-11 05:56:38 - CEBinaryAccuracyEvaluator: Evaluating the model on dev dataset after epoch 3:
2022-06-11 05:56:50 - Accuracy: 83.74


Iteration:   0%|          | 0/17029 [00:00<?, ?it/s]

2022-06-11 06:22:44 - CEBinaryAccuracyEvaluator: Evaluating the model on dev dataset in epoch 4 after 10000 steps:
2022-06-11 06:22:56 - Accuracy: 83.75
2022-06-11 06:41:07 - CEBinaryAccuracyEvaluator: Evaluating the model on dev dataset after epoch 4:
2022-06-11 06:41:19 - Accuracy: 83.84
2022-06-11 06:41:19 - Save model to output/cross-encoder_hlf-rbtl3-2022-06-11


Iteration:   0%|          | 0/17029 [00:00<?, ?it/s]

2022-06-11 07:07:10 - CEBinaryAccuracyEvaluator: Evaluating the model on dev dataset in epoch 5 after 10000 steps:
2022-06-11 07:07:22 - Accuracy: 83.75


In [ ]:
test_evaluator1 = CEBinaryAccuracyEvaluator.from_input_examples(test_samples, name='test1')
test_evaluator2 = CEBinaryClassificationEvaluator.from_input_examples(test_samples, name='test2')
test_evaluator1(model, output_path=model_save_path)
test_evaluator2(model, output_path=model_save_path)

In [ ]:
os.system("shutdown")